In [1]:
import pandas as pd
import numpy as np 

# To access the database
from modules.postgres import AppSettings, PostgresConnector
from queries.queries import insert_query

# To access the weather API at https://open-meteo.com/en/docs/historical-weather-api
import openmeteo_requests
import requests_cache
from retry_requests import retry

Set up Database Connection 

In [9]:
# Set up the connection to the database 
settings = AppSettings(
    DBNAME="senior-project-db",
    USER="postgre",
    PASSWORD="BxQdWkTb3bkU6s1tkfi8",
    HOST="senior-project-db.cfc84082wdma.us-east-1.rds.amazonaws.com",
)

# Create client that is connected to the database 
client = PostgresConnector(settings)

Census Blocks 

In [ ]:
# Load the csv containing census block data for Cleveland 
census_blocks = pd.read_csv('census-block-locations.csv')

# Convert types to be properly stored in database
census_blocks["geoid"] = census_blocks["geoid"].astype(str)

# Set the database table name, the columns, and values to be inserted 
table_name = "census_blocks"
cols = ",".join(list(census_blocks.columns))
values = [tuple(block) for block in census_blocks.to_numpy()]

# Insert values
try: 
    insert_query(client, table_name, cols, values)
except Exception as e: 
    print(f"Error when inserting data: {e}")

Crime Data

In [ ]:
# read in csv of crime data 
crime_data = pd.read_csv("crime_incidents.csv")

# Clean up the columns that will be inserted 
crime_data = crime_data.dropna(subset=["LAT", "LON"]) # drop rows that don't have a location 
crime_data = crime_data.drop(["x", "y"], axis=1) # not including the x and y columns, using lat and lon for location 
crime_data = crime_data.loc[crime_data["OffenseYear"] >= 2018] # include only crimes from 2018 to present

# Convert ids to strings to conform with database 
crime_data["OBJECTID"] = crime_data["OBJECTID"].astype(str) 
crime_data["PrimaryKey"] = crime_data["PrimaryKey"].astype(str)
crime_data["CENSUS_TRACT_GEOID"] = crime_data["CENSUS_TRACT_GEOID"].astype(str)
crime_data["CENSUS_BG_GEOID"] = crime_data["CENSUS_BG_GEOID"].astype(str)
crime_data["CENSUS_BLOCK_GEOID"] = crime_data["CENSUS_BLOCK_GEOID"].astype(str)

# Fill missing zip codes with zero and then convert to int
crime_data["Zip"] = crime_data["Zip"].fillna(0)
crime_data["Zip"] = crime_data["Zip"].astype(int)

# Set the database table name, the columns, and values to be inserted 
table_name = "crimes"
cols = "object_id, primary_key, case_number, district, ucr_desc, offense_year, time_group, reported_date, offense_month, offense_day, time_block, dow_name, dow, hour_of_day, days_ago, offense_date, statute, zip, stat_desc, address_public, std_parcelpin, ward, city, census_tract, census_tract_geoid, census_block_group, census_bg_geoid, census_block, census_block_geoid, lat, lon"
crime_data_values = [tuple(data) for data in crime_data.to_numpy()]

# Insert values
try: 
    insert_query(client, table_name, cols, crime_data_values)
except Exception as e: 
        print(f"Error when inserting data: {e}")

Weather Data 

In [22]:
# Can only hit the API a few items per hour and there is a daily limit to consider
# Create list of latitudes and longitudes to be fetched
lat_to_insert = census_blocks["lat_center"].tolist()[6600:]
lon_to_insert = census_blocks["lon_center"].tolist()[6600:]
start_date = "2018-01-01"
end_date = "2024-10-01"

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": lat_to_insert,
    "longitude": lon_to_insert,
    "start_date": start_date,
    "end_date": end_date,
    "daily": [
        "temperature_2m_max",
        "temperature_2m_min",
        "daylight_duration",
        "precipitation_sum",
        "precipitation_hours",
    ],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "mph",
    "precipitation_unit": "inch",
    "timezone": "America/New_York",
}

try: 
    responses = openmeteo.weather_api(url, params=params)
except Exception as e: 
    print(f"Error when retrieving data: {e}")

In [23]:
# Table and columns in the database
table_name = "historical_weather"
cols = "lat, lon, date, temp_max, temp_min, daylight_duration, precipitation_sum, precipitation_hours"

# Go through the responses and insert into database
for i in range(len(responses)):
    response = responses[0]

    # Retrieve daily data
    daily = response.Daily()

    # Get the data range 
    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left",
        )
    }

    # Assign each of the selected variables 
    daily_data["temperature_2m_max"] = daily.Variables(0).ValuesAsNumpy()
    daily_data["temperature_2m_min"] = daily.Variables(1).ValuesAsNumpy()
    daily_data["daylight_duration"] = daily.Variables(2).ValuesAsNumpy()
    daily_data["precipitation_sum"] = daily.Variables(3).ValuesAsNumpy()
    daily_data["precipitation_hours"] = daily.Variables(4).ValuesAsNumpy()

    # Convert the values to list of tuples 
    weather_data = [tuple(data) for data in pd.DataFrame(daily_data).to_numpy()]
    # Add the latitude and longitude to all of the tuples 
    lat = lat_to_insert[i]
    lon = lon_to_insert[i]
    weather_data_with_loc = [(lat, lon) + t for t in weather_data]

    # Insert into database 
    try: 
        insert_query(client, table_name, cols, weather_data_with_loc)
    except Exception as e: 
        print(f"Error when inserting data: {e}")

Sports Data

In [10]:
# read in csv of sports data 
sport_data = pd.read_csv("past_sports_schedules_census_blocks.csv")[["Date", "Location", "Home Team", "latitude", "longitude", "nearest_geo_id"]]


table_name = "sports_events"
cols = "date, location, home_team, lat, lon, geoid"
sport_data_values = [tuple(data) for data in sport_data.to_numpy()]

# Insert values
try: 
    insert_query(client, table_name, cols, sport_data_values)
except Exception as e: 
        print(f"Error when inserting data: {e}")

Accessing crime data with weather data 

In [29]:
# Join statement to get crime data with weather data 
query = """
WITH census_block_weather AS (
	SELECT geoid, historical_weather.* FROM census_blocks
	JOIN historical_weather ON census_blocks.lat_center = historical_weather.lat AND census_blocks.lon_center = historical_weather.lon
)
SELECT * FROM crimes 
JOIN census_block_weather ON crimes.census_block_geoid = census_block_weather.geoid AND date(crimes.offense_date) = census_block_weather.date
"""

# Read the data from the database 
result = client.read(query)